<a href="https://colab.research.google.com/github/JColeman1550/BASEBALL.github.io/blob/main/Copy_of_CATCHER_ARM_EVALUATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!pip install pybaseball[catchers]
from pybaseball import statcast
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import pybaseball as pyb

Pandas is a powerful data manipulation and analysis library. It provides data structures and functions to work with structured data, such as data frames. I am using it to handle and manipulate my data throughout the code.

NumPy is a fundamental package for Python. It provides support for large, multi-dimensional arrays and a collection of mathematical functions. I am using it for numerical computations in my code.

sklearn.cluster.KMeans: This package is part of scikit-learn, which is a popular machine learning library in Python. The KMeans class is used for clustering data points into groups or clusters based on their similarity. I am utilizing this package to perform k-means clustering on your catcher arm data.

pybaseball (!pip install pybaseball[catchers]): Pybaseball is a Python library that provides easy access to baseball data, including statistics and player information. By installing the catchers module, I gain access to specific functionalities related to catcher statistics.

 StandardScaler is a class that provides methods to standardize or normalize data by scaling it to a mean of 0 and a standard deviation of 1. I am using it to scale my catcher arm data before performing clustering analysis, ensuring that all features are on a similar scale.


In [ ]:
data = pyb.statcast(start_dt='2022-01-01', end_dt='2022-12-31')

This is a large query, it may take a moment to complete


/usr/local/lib/python3.10/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


100%|██████████| 246/246 [05:08<00:00,  1.26s/it]


This line of code retrieves baseball data using the Pybaseball library. Specifically, it calls the statcast function to retrieve data from a specified time period. In this case, the start_dt parameter is set to '2022-01-01', indicating that the data collection should start from January 1, 2022. Similarly, the end_dt parameter is set to '2022-12-31', indicating that the data collection should end on December 31, 2022. Offseason dates are skipped

The statcast function collects a wide range of information related to baseball plays, including pitch velocity, launch angle, exit velocity, and various other metrics. By specifying the start and end dates, I am ensuring that the data collected is limited to the desired time period.

In [ ]:
pyb.statcast_catcher_poptime

<function pybaseball.statcast_fielding.statcast_catcher_poptime(year: int, min_2b_att: int = 5, min_3b_att: int = 0) -> pandas.core.frame.DataFrame>

The code pyb.statcast_catcher_poptime is used to retrieve catcher-specific statistical data related to pop times using the Pybaseball library.
By calling pyb.statcast_catcher_poptime, I can access data specifically focused on catchers and their pop times. This data includes information such as the catcher's name and the corresponding pop time for each throw. 

In [ ]:
catcher_stats = pyb.statcast_fielding.statcast_catcher_poptime(year= 2022, min_2b_att = 0, min_3b_att = 0)

The code: catcher_stats = pyb.statcast_fielding.statcast_catcher_poptime(year=2022, min_2b_att=0, min_3b_att=0) retrieves catcher-specific statistical data related to pop times using the Pybaseball library. This code specifically fetches the pop time data for the year 2022.
The additional parameters min_2b_att and min_3b_att are used to filter the data based on the minimum number of attempts at second base (2B) and third base (3B) respectively. In this case, both parameters are set to 0, meaning that there is no minimum threshold for the number of attempts required. 


In [ ]:
catcher_stats.head()

,catcher,player_id,team_id,age,maxeff_arm_2b_3b_sba,exchange_2b_3b_sba,pop_2b_sba_count,pop_2b_sba,pop_2b_cs,pop_2b_sb,pop_3b_sba_count,pop_3b_sba,pop_3b_cs,pop_3b_sb
0,Matt Thaiss,NaN,108.0,27.0,83.9,0.79,5,1.99,1.82,2.03,0,NaN,NaN,NaN
1,J.T. Realmuto,NaN,143.0,31.0,87.6,0.67,44,1.82,1.83,1.82,1,1.40,NaN,1.40
2,Elias Díaz,NaN,115.0,31.0,84.8,0.71,28,1.94,1.87,1.98,5,1.43,1.4,1.44
3,Will Smith,NaN,119.0,27.0,83.6,0.71,32,1.93,1.88,1.94,2,1.51,NaN,1.51
4,Chuckie Robinson,NaN,113.0,27.0,86.3,0.73,11,1.90,1.89,1.90,1,1.54,NaN,1.54


In [ ]:
catcher_stats = catcher_stats[['catcher', 'maxeff_arm_2b_3b_sba']]

The line of code: catcher_stats = catcher_stats[['catcher', 'maxeff_arm_2b_3b_sba']] is used to select and retain specific columns from the catcher_stats DataFrame.
By using double square brackets and specifying ['catcher', 'maxeff_arm_2b_3b_sba'] within them, I am selecting only these two columns from the original DataFrame and assigning the updated DataFrame back to the variable catcher_stats.

In [ ]:
catcher_stats = catcher_stats.dropna()

The dropna() function is a pandas method that checks each row in the DataFrame and removes any row that has at least one missing value. I am using this to clean the data

In [ ]:
scaler = StandardScaler()
catcher_stats_scaled = scaler.fit_transform(catcher_stats[['maxeff_arm_2b_3b_sba']])

The code scaler = StandardScaler() creates an instance of the StandardScaler class from the scikit-learn library.
The catcher_stats[['maxeff_arm_2b_3b_sba']] expression selects the 'maxeff_arm_2b_3b_sba' column from the catcher_stats DataFrame. By passing this column as the input to fit_transform(), the scaler calculates the mean and standard deviation of this specific feature and transforms it accordingly.
This code applies the StandardScaler to the 'maxeff_arm_2b_3b_sba' column of the catcher_stats DataFrame, standardizing the values and storing the transformed data in a new variable "catcher_stats_scaled".

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(catcher_stats_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


n_clusters is set to 3, meaning that the algorithm will try to find three distinct clusters within the data.
The random_state parameter is set to 0 to ensure reproducibility of the results. By setting a specific random state value, the clustering process will yield the same results each time the code is executed, which is useful for consistency and comparison.
The fit() method is called on the KMeans object (kmeans) to fit the model to the scaled data. This step performs the actual clustering process, where the algorithm iteratively assigns data points to the nearest cluster centroid and updates the centroids based on the assigned points.


In [ ]:
catcher_stats['cluster'] = kmeans.labels_

catcher_stats['cluster'] = kmeans.labels_ adds a new column called 'cluster' to the catcher_stats DataFrame and assigns the cluster labels obtained from the k-means clustering algorithm to each data point.
By assigning these cluster labels to the 'cluster' column of the catcher_stats DataFrame, I associated each catcher's data point with the cluster it belongs to.

In [ ]:
for cluster in range(kmeans.n_clusters):
    print("Cluster ", cluster)
    print(catcher_stats[catcher_stats['cluster'] == cluster])
    print("\n")

Cluster  0
               catcher  maxeff_arm_2b_3b_sba  cluster
0          Matt Thaiss                  83.9        0
3           Will Smith                  83.6        0
5          Nick Fortes                  80.7        0
7      Meibrys Viloria                  83.2        0
8       Garrett Stubbs                  83.5        0
9       Gabriel Moreno                  81.8        0
15   Christian Vázquez                  80.4        0
17    Tyler Stephenson                  84.0        0
18          José Godoy                  79.8        0
21        Payton Henry                  81.8        0
22        Brian Serven                  81.6        0
23         MJ Melendez                  83.0        0
24     Kyle Higashioka                  80.1        0
25     Anthony Bemboom                  82.5        0
28       Austin Hedges                  82.1        0
29          Eric Haase                  83.2        0
31         Austin Nola                  81.4        0
37        Jose He

This loop allows me to iterate through each cluster, print the cluster number, and display the catchers' information within that cluster. 